In [1]:
# CELLULE 1 — Imports & paths

# ===========================================================
# Notebook 03 — Preprocessing FINAL (Resize + CLAHE)
# Dataset unifié, splits gérés par CSV
# ===========================================================

import os
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm

BASE_DIR = Path("/home/fabrice/Boneage")

# Dossier des images DEJA MASQUÉES
MASKED_DIR = BASE_DIR / "boneage_segmented_eff_unet"

# CSV v4 (stratification combinée)
TRAIN_CSV = BASE_DIR / "boneage_train_final_v3.csv"
VAL_CSV   = BASE_DIR / "boneage_val_v3.csv"
TEST_CSV  = BASE_DIR / "boneage_test_v3.csv"

# Dossier final UNIQUE
OUT_DIR = BASE_DIR / "preprocessed_v3"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Résolution finale (300 recommandé pour V2B3, 384 pour V2L)
IMG_SIZE = 300

print("Images masquées :", MASKED_DIR)
print("Dossier final   :", OUT_DIR)
print("IMG_SIZE        :", IMG_SIZE)


Images masquées : /home/fabrice/Boneage/boneage_segmented_eff_unet
Dossier final   : /home/fabrice/Boneage/preprocessed_v3
IMG_SIZE        : 300


In [2]:
# CELLULE 2 — Charger les CSV (uniquement pour vérifier les IDs présents)

train_df = pd.read_csv(TRAIN_CSV)
val_df   = pd.read_csv(VAL_CSV)
test_df  = pd.read_csv(TEST_CSV)

# Ensemble de tous les IDs du dataset
all_ids = pd.concat([train_df["id"], val_df["id"], test_df["id"]]).unique()

print("Nombre total d'IDs :", len(all_ids))
print("Exemples :", list(all_ids[:10]))


Nombre total d'IDs : 11325
Exemples : [12380, 9535, 12319, 10433, 9223, 14637, 13048, 7747, 1796, 7881]


In [3]:
# CELLULE 3 — Fonctions (load → resize → CLAHE)

def load_masked_image(img_id):
    path = MASKED_DIR / f"{img_id}.png"
    return cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)

def apply_clahe(img):
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
    return clahe.apply(img)

def preprocess_image(img_id):
    # 1) Charger l’image masquée
    img = load_masked_image(img_id)
    if img is None:
        return None

    # 2) Resize
    img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)

    # 3) CLAHE
    img_final = apply_clahe(img_resized)

    return img_final


In [4]:
# CELLULE 4 — Traitement de toutes les images

missing = []

print("=== Préprocessing de TOUTES les images masquées ===")

for img_id in tqdm(all_ids):
    processed = preprocess_image(img_id)

    if processed is None:
        missing.append(img_id)
        continue

    # Sauvegarde
    out_path = OUT_DIR / f"{img_id}.png"
    cv2.imwrite(str(out_path), processed)

print("\nPréprocessing terminé.")


=== Préprocessing de TOUTES les images masquées ===


100%|█████████████████████████████████████████████████████████████████████████████| 11325/11325 [03:10<00:00, 59.30it/s]


Préprocessing terminé.


In [5]:
# CELLULE 5 — Rapport final

print("\n=== RAPPORT FINAL ===")

if len(missing) == 0:
    print(" Toutes les images ont été préprocessées avec succès.")
else:
    print(" IMAGES MANQUANTES :", missing)

print("\nImages finales prêtes dans :", OUT_DIR)
print("Nombre total d'images créées :", len(list(OUT_DIR.glob('*.png'))))



=== RAPPORT FINAL ===
 Toutes les images ont été préprocessées avec succès.

Images finales prêtes dans : /home/fabrice/Boneage/preprocessed_v3
Nombre total d'images créées : 11325
